In [1]:
from flask import Flask, request
import json
from textblob import TextBlob

import pandas as pd

In [2]:
inverted_index = {}
forward_index = []
documents_id = []

In [3]:
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World from Scissors!"


@app.route('/about')
def about():
    return 'The about page'
        

@app.route('/ranging', methods=["POST"])
def ranging():
    """
    Returns:
    List with dictionaries according to ranged documents ID as json file
    """
    list_of_dicts = request.json 
    documents = [Document(**doc) for doc in list_of_dicts]
    return json.dumps([doc.to_dict() for doc in documents])
    
    
@app.route('/snippets', methods=["POST"])
def get_snippets():
    """
    Returns:   
    """     
    search_words = request.json["search_word"]
    result = search_one(search_words)
    dicts = []
    for doc_id in result:
        val = str(df[df.id==doc_id].text.values)
        dicts.append({str(doc_id): val})
    return json.dumps(dicts[0])


@app.route('/return_page', methods=["POST"])
def return_page():
    """
    Returns:
    
    """
    list_of_dicts = request.json 
    documents = [Document(**doc) for doc in list_of_dicts]
    result = [doc.to_dict() for doc in documents]
    
    return "html"



@app.route('/indexator',  methods=["POST"])
def add_to_index():
    """
    Returns:
    
    """
    document = request.json 
    add_inverted_index(document)
    add_forward_index(document)
   
    return "added to index"


@app.route("/search", methods=["POST"])
def search():
    search_words = request.json["search_word"]
    
    result = search_one(search_words)
    
    return json.dumps(result)

In [4]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13522)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13522/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2018 18:40:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2018 18:40:24] "GET /favicon.ico HTTP/1.1" 404 -


In [88]:
def snippet(doc_id):
    return df[df.id==doc_id].text.values

In [89]:
snippet(394513)

array(['Должностные обязанности: * Грамотная выкладка товара Чиртон * Контроль ценников, сроков годности * Размещение рекламных материалов * Предоставление фотоотчета по эл.почте, whatsapp Требования: Опыт работы приветствуется Наличие л/а Предпочтительны кандидаты без опыта-всему с нуля обучаем! Водительские права категории: B. Мы предлагаем: * Официальное трудоустройство * гсм входит в заработную плату Полный рабочий день Разъездного характера'],
      dtype=object)

In [61]:
def add_inverted_index(document):
    tokens = TextBlob(document["text_searchable"]).words
    
    if document["id"] not in documents_id:
        documents_id.append(document["id"])
        for token in tokens:
            if token in inverted_index.keys():
                inverted_index[token].append({"doc_id": int(document["id"])})
            else:
                inverted_index[token] = [{"doc_id": int(document["id"])}]
          

In [62]:
def add_forward_index(document):
    if document["id"] not in documents_id:
        documents_id.append(document["id"])
        forward_index.append({"doc_id": int(document["id"]), "text": document["text"]})

In [90]:
def search_one(words):
    idx = []
    words = words.split(" ")
      
    for word in words:
        if word in inverted_index.keys():
            idx.append(set( [doc["doc_id"] for doc in inverted_index[word]]))     
    
    indexes = idx[0]
    for doc_set in idx:
        indexes = indexes.intersection(doc_set)
    
    return list(indexes)

In [69]:
inverted_index = {}
forward_index = []
documents_id = []

In [76]:
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World from Scissors!"


@app.route('/about')
def about():
    return 'The about page'
        

@app.route('/ranging', methods=["POST"])
def ranging():
    """
    Returns:
    List with dictionaries according to ranged documents ID as json file
    """
    list_of_dicts = request.json 
    documents = [Document(**doc) for doc in list_of_dicts]
    return json.dumps([doc.to_dict() for doc in documents])
    
    
@app.route('/snippets', methods=["POST"])
def get_snippets(search_res):
    """
    Returns:   
    """     
    list_of_dicts = request.json 
    snippets = []
    for val in search_res:
        curr_snippet = df[df.id == val["doc_id"]]
        snippets.appen(curr_snippet)
        
        
    return json.dumps(search_res)


@app.route('/return_page', methods=["POST"])
def return_page():
    """
    Returns:
    
    """
    list_of_dicts = request.json 
    documents = [Document(**doc) for doc in list_of_dicts]
    result = [doc.to_dict() for doc in documents]
    
    return "html"



@app.route('/indexator',  methods=["POST"])
def add_to_index():
    """
    Returns:
    
    """
    document = request.json  
    add_inverted_index(document)
    add_forward_index(document)
   
    return "added to index"

@app.route("/search", methods=["POST"])
def search():
    search_word = request.json["search_word"]
    
    result = search_one(search_word)
    return json.dumps(result)

In [77]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13522)

 * Running on http://0.0.0.0:13522/ (Press CTRL+C to quit)
[2018-11-08 18:49:51,595] ERROR in app: Exception on /search [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_

In [53]:
def add_inverted_index(document):
    tokens = TextBlob(document["text_searchable"]).words
    
    if document["id"] not in documents_id:
        documents_id.append(document["id"])
        for token in tokens:
            if token in inverted_index.keys():
                inverted_index[token].append({"doc_id": int(document["id"])})
            else:
                inverted_index[token] = [{"doc_id": int(document["id"])}]

In [52]:
def add_forward_index(document):
    if document["id"] not in documents_id:
        documents_id.append(document["id"])
        forward_index.append({"doc_id": document["id"], "text": document["text"]})

In [74]:
%%time
for i in range(len(df)):
    doc = df.loc[i].to_dict()
    add_inverted_index(doc)
    add_forward_index(doc)

KeyboardInterrupt: 

In [51]:
def search_one(word):
    idx = []
       
    if word in inverted_index.keys():
        idx.append(inverted_index[word])
    else:
        idx.append("Not found word " + word)
        
    return idx